<a href="https://colab.research.google.com/github/cheolhakja/fine-dust-prediction/blob/main/gnn/create_adjacent_weighted_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import haversine_distances
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# CSV 파일 읽기
df = pd.read_csv('/content/drive/MyDrive/graduation_project/도시_도로변_입체_측정소위치.csv', encoding='utf8')
df = df[df['sido']=='서울'].copy()

# 고유한 측정소 추출
stations = df[['stationName', 'dmX', 'dmY']].drop_duplicates()

# 좌표 추출 (위도: dmX, 경도: dmY)
coords_deg = stations[['dmX', 'dmY']].values

# 라디안으로 변환
coords_rad = np.radians(coords_deg)

# 쌍별 거리 계산 (라디안 단위)
distances_rad = haversine_distances(coords_rad)

# 킬로미터로 변환 (지구 반지름 ≈ 6371km)
distances_km = distances_rad * 6371

# σ 값 설정 (서울의 공간적 규모를 고려하여 3km로 설정)
sigma = 3  # 단위: km

# 지수 가중치 인접 행렬 계산
adj_matrix = np.exp(-distances_km / sigma)

# 대각선 요소를 0으로 설정 (자기 연결 제거)
np.fill_diagonal(adj_matrix, 0)

# 결과 저장
np.savetxt('adj_matrix.txt', adj_matrix, delimiter=',')

# 측정소 순서 출력 (행렬의 행/열이 어떤 측정소에 해당하는지 확인용)
print("측정소 순서:", stations['stationName'].values)

# σ 선택 검증을 위한 거리 분포 확인 (선택적)
distances_flat = distances_km[np.triu_indices(len(stations), k=1)]
median_distance = np.median(distances_flat)
print(f"중앙 거리: {median_distance:.2f} km")

측정소 순서: ['중구' '한강대로' '종로구' '청계천로' '종로' '용산구' '광진구' '성동구' '강변북로' '중랑구' '동대문구' '홍릉로'
 '성북구' '정릉로' '도봉구' '은평구' '서대문구' '마포구' '신촌로' '강서구' '공항대로' '구로구' '영등포구'
 '영등포로' '동작구' '동작대로 중앙차로' '관악구' '강남구' '서초구' '도산대로' '강남대로' '송파구' '강동구'
 '천호대로' '금천구' '시흥대로' '강북구' '양천구' '노원구' '화랑로' '관악산' '남산' '마포아트센터' '북한산'
 '자연사박물관' '서울숲' '세곡' '올림픽공원' '항동' '행주']
중앙 거리: 11.44 km
